# ControlNet Floorplan Generator - Testing Notebook

**Test your trained model with upload/drawing before deploying!**

## What This Does:
- ✅ Load your trained ControlNet model
- ✅ Test with dataset samples (check overfitting)
- ✅ Upload your own segmentation masks
- ✅ Draw custom layouts interactively
- ✅ Generate variations & explore parameters
- ✅ Save results for your report

## Setup Instructions:
1. **Runtime → Change runtime type → T4 GPU**
2. Run cells in order from top to bottom
3. Wait for model loading (~2-3 minutes)
4. Start testing!

---

## Step 1: Install Dependencies

In [ ]:
# === INSTALLATION - FIXED FOR COMPATIBILITY ===
# This fixes the 'cached_download' import error

import sys
print(f"Python version: {sys.version}\n")

# Clean install to avoid conflicts
print("Cleaning old packages...")
!pip uninstall -y diffusers transformers accelerate huggingface_hub -q

# Install latest compatible versions
print("Installing compatible packages...")
!pip install -q diffusers>=0.27.0
!pip install -q transformers>=4.40.0
!pip install -q accelerate>=0.27.0
!pip install -q huggingface_hub>=0.20.0
!pip install -q gradio>=4.20.0
!pip install -q datasets

# Verify installation
print("\nVerifying installation...")
import torch
try:
    from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
    from transformers import __version__ as trans_ver
    from diffusers import __version__ as diff_ver
    print(f"Diffusers: {diff_ver}")
    print(f"Transformers: {trans_ver}")
    print(f"PyTorch: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
    print("\nInstallation complete! You can proceed.")
except Exception as e:
    print(f"❌ Error: {e}")
    print("\nIf you see an error, click Runtime → Restart runtime, then re-run this cell.")


## Step 2: Load Your Trained Model

In [ ]:
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset
import gradio as gr
import random

print("Loading your trained ControlNet model...")
print("This takes 2-3 minutes on first load")

# Load your trained ControlNet
controlnet = ControlNetModel.from_pretrained(
    "aqhareus/controlnet-floorplan-final",
    torch_dtype=torch.float16
)

# Load Stable Diffusion pipeline
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16,
    safety_checker=None
)

# Move to GPU and optimize memory
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = pipe.to(device)
pipe.enable_model_cpu_offload()
pipe.enable_attention_slicing()

print(f"\nModel loaded successfully on {device}!")
print(f"Memory optimizations enabled")

## Step 3: Define Room Colors & Generation Function

In [ ]:
# Room color palette (matching your dataset)
# Room color palette - CORRECTED TO MATCH ACTUAL DATASET
# These are the exact RGB values your model was trained on!
# Room color palette - UPDATED from dataset analysis
ROOM_COLORS = {
    "Living Room": (154, 255, 0),      # Lime/Yellow-Green
    "Bedroom": (254, 154, 0),          # Orange
    "Kitchen": (154, 255, 0),          # Lime/Yellow-Green (same as Living Room)
    "Bathroom": (0, 155, 255),         # Cyan/Light Blue
    "Closet": (99, 80, 71),            # Brown/Gray
    "Corridor": (49, 99, 155),         # Dark Blue
    "Toilet": (0, 0, 0),               # Black
}

print("Room Color Palette:")
for room, color in ROOM_COLORS.items():
    print(f"   {room:15s} → RGB{color}")

def generate_floorplan(
    segmentation_mask,
    prompt="a clean architectural floorplan with walls and rooms",
    num_inference_steps=20,
    controlnet_conditioning_scale=1.0,
    seed=42,
    show_comparison=True
):
    """
    Generate a floorplan from a segmentation mask.

    Args:
        segmentation_mask: PIL Image or numpy array (RGB)
        prompt: Text description
        num_inference_steps: Quality (10-50, higher=better/slower)
        controlnet_conditioning_scale: Layout strictness (0.5-2.0)
        seed: Random seed for reproducibility
        show_comparison: Display input vs output

    Returns:
        Generated PIL Image
    """
    # Prepare image
    if isinstance(segmentation_mask, np.ndarray):
        segmentation_mask = Image.fromarray(segmentation_mask)

    segmentation_mask = segmentation_mask.convert("RGB").resize((512, 512), Image.LANCZOS)

    # Generate
    generator = torch.Generator(device=device).manual_seed(seed)

    output = pipe(
        prompt=prompt,
        image=segmentation_mask,
        num_inference_steps=num_inference_steps,
        controlnet_conditioning_scale=controlnet_conditioning_scale,
        generator=generator,
        guidance_scale=7.5
    )

    generated = output.images[0]

    # Display comparison
    if show_comparison:
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        axes[0].imshow(segmentation_mask)
        axes[0].set_title("Input Segmentation", fontsize=14, fontweight='bold')
        axes[0].axis('off')

        axes[1].imshow(generated)
        axes[1].set_title("Generated Floorplan", fontsize=14, fontweight='bold')
        axes[1].axis('off')

        plt.tight_layout()
        plt.show()

    return generated

print("\nGeneration function ready!")

## Step 4: Test with Dataset Samples (Overfitting Check)

**This tests if your model is overfitting by comparing:**
- Input segmentation
- Generated floorplan (your model)
- Ground truth (training data)

**What to look for:**
- ✅ Generated matches input layout structure
- ✅ Generated is NOT identical to ground truth (overfitting sign)
- ✅ Generated has realistic architectural details

In [ ]:
# Load dataset
print("Loading test samples from dataset...")
dataset = load_dataset("Qistinasofea/floorplan-aligned-strict", split="train")

# Test 3 random samples
test_indices = random.sample(range(len(dataset)), 3)
print(f"Testing samples: {test_indices}\n")

for i, idx in enumerate(test_indices):
    sample = dataset[idx]

    print(f"{'='*70}")
    print(f"Sample {i+1}/3 (Dataset index: {idx})")
    print(f"Caption: {sample['captions']}")
    print(f"{'='*70}")

    # Generate
    generated = generate_floorplan(
        segmentation_mask=sample['colors'],
        prompt=sample['captions'],
        seed=42,
        show_comparison=False
    )

    # Display: Input → Generated → Ground Truth
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    axes[0].imshow(sample['colors'])
    axes[0].set_title("Input Segmentation", fontsize=12, fontweight='bold')
    axes[0].axis('off')

    axes[1].imshow(generated)
    axes[1].set_title("Generated (Your Model)", fontsize=12, fontweight='bold')
    axes[1].axis('off')

    axes[2].imshow(sample['plans'])
    axes[2].set_title("Ground Truth", fontsize=12, fontweight='bold')
    axes[2].axis('off')

    plt.suptitle(f"Dataset Sample {idx} - Overfitting Check", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

    print("\nEvaluation Questions:")
    print(" Does generated match input layout?")
    print(" Is generated TOO similar to ground truth? (overfitting)")
    print(" Are walls/rooms in correct positions?\n")

print("\n✅ Dataset testing complete!")

## Step 5: Create Custom Layouts Programmatically

Test your model on layouts it has NEVER seen before!

In [ ]:
def create_simple_layout(rooms_config, size=512):
    """
    Create segmentation mask with ABSOLUTELY NO BORDERS.

    Uses numpy array manipulation for pixel-perfect control.
    PIL's draw.rectangle() has anti-aliasing artifacts - this doesn't!
    """
    import numpy as np

    # Create black background as numpy array
    img_array = np.zeros((size, size, 3), dtype=np.uint8)

    # Fill rooms by directly setting pixel values (NO BORDERS!)
    for room_name, x, y, w, h in rooms_config:
        color = ROOM_COLORS.get(room_name, (128, 128, 128))

        # Direct pixel assignment - perfect fills, zero borders
        img_array[y:y+h, x:x+w] = color

    # Convert to PIL Image
    return Image.fromarray(img_array, 'RGB')


# Test Layout 1: Simple 2-Bedroom Apartment
print("Test 1: Simple 2-Bedroom Apartment (PERFECT - No Borders)")
layout1 = create_simple_layout([
    ("Living Room", 0, 0, 256, 256),      # Top-left
    ("Bedroom", 256, 0, 256, 256),        # Top-right (starts at 256!)
    ("Living Room", 0, 256, 128, 256),    # Bottom-left
    ("Bathroom", 128, 256, 128, 256),     # Bottom-middle (starts at 128!)
    ("Bedroom", 256, 256, 256, 256)       # Bottom-right (starts at 256!)
])
generated1 = generate_floorplan(layout1, seed=457)

# Test Layout 2: Studio Apartment
print("\nTest 2: Studio Apartment")
layout2 = create_simple_layout([
    ("Living Room", 0, 0, 512, 256),      # Full width top
    ("Corridor", 0, 256, 256, 256),        # Left half
    ("Bathroom", 256, 256, 256, 256)      # Right half (starts at 256!)
])
generated2 = generate_floorplan(layout2, seed=412)

# Test Layout 3: Complex Multi-Room House
print("\nTest 3: Complex Multi-Room House")
layout3 = create_simple_layout([
    # Row 1: Living Room (230px) + Kitchen (282px) = 512px ✓
    ("Living Room", 0, 0, 230, 170),
    ("Kitchen", 230, 0, 282, 170),

    # Row 2: Corridor (20px) + Kitchen (210px) + Bedroom (282px) = 512px ✓
    ("Corridor", 0, 170, 20, 170),
    ("Kitchen", 20, 170, 210, 170),
    ("Bedroom", 230, 170, 282, 170),

    # Row 3: Bedroom (140px) + Toilet (70px) + Bedroom (160px) + Bathroom (142px) = 512px ✓
    ("Bedroom", 0, 340, 140, 172),
    ("Toilet", 140, 340, 70, 172),
    ("Bedroom", 210, 340, 160, 172),
    ("Bathroom", 370, 340, 142, 172)
])
generated3 = generate_floorplan(layout3, seed=212)

print("\n✅ Custom layouts tested - ZERO borders!")
print("✅ Using numpy for pixel-perfect control")


## Step 6: Test Diversity (Multiple Seeds)

**Check if your model generates diverse outputs or just memorized patterns**

In [ ]:
import wandb
import matplotlib.pyplot as plt

# If you are in Colab / notebook and not logged in yet:
# wandb.login()

# ---- W&B init ----
wandb.init(
    project="AI54-Floorplan",      # change
    name="diversity_test_seeds_layout_latest", # change
    config={
        "layout": "layout2",
        "seeds": [42, 123, 456, 789],
    }
)

print("Generating 4 variations with different seeds...\n")

test_layout = layout2  # Use studio apartment
seeds = [42, 123, 456, 789]

# Log the input segmentation once
wandb.log({"input_segmentation": wandb.Image(test_layout, caption="Input Segmentation")})

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

# Show input
axes[0].imshow(test_layout)
axes[0].set_title("Input Segmentation", fontsize=12, fontweight="bold")
axes[0].axis("off")

# Generate variations + log to W&B
for i, seed in enumerate(seeds):
    generated = generate_floorplan(
        segmentation_mask=test_layout,
        seed=seed,
        show_comparison=False
    )

    # Show in the grid
    axes[i + 1].imshow(generated)
    axes[i + 1].set_title(f"Seed={seed}", fontsize=12, fontweight="bold")
    axes[i + 1].axis("off")

    # Log each generated image to W&B
    wandb.log({
        "seed": seed,  # useful for filtering
        f"generated/seed_{seed}": wandb.Image(generated, caption=f"Generated (seed={seed})")
    })

    # (Optional) Log side-by-side comparison (input + output) as a single image
    panel_fig, panel_ax = plt.subplots(1, 2, figsize=(8, 4))
    panel_ax[0].imshow(test_layout)
    panel_ax[0].set_title("Input")
    panel_ax[0].axis("off")
    panel_ax[1].imshow(generated)
    panel_ax[1].set_title(f"Output (seed={seed})")
    panel_ax[1].axis("off")
    plt.tight_layout()

    wandb.log({f"comparison/seed_{seed}": wandb.Image(panel_fig)})
    plt.close(panel_fig)

# Hide last subplot
axes[5].axis("off")

plt.suptitle("Diversity Test: Same Input, Different Seeds", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

print("\nEvaluation:")
print(" Good: Outputs show variation (different details, styles)")
print(" Bad: All outputs nearly identical (overfitting sign)")

wandb.finish()


## Step 7: Explore Conditioning Scale

**Test how strictly the model follows your input layout**

In [ ]:
import wandb
import matplotlib.pyplot as plt

# If not logged in yet:
# wandb.login()

# ---- W&B init ----
wandb.init(
    project="AI54-Floorplan",          # change if needed
    name="conditioning_scale_test_layout2",  # change if needed
    config={
        "layout": "layout2",
        "conditioning_scales": [0.5, 0.75, 1.0, 1.5, 2.0],
        "seed": 42
    }
)

print("Testing conditioning scale (layout strictness)...\n")

scales = [0.5, 0.75, 1.0, 1.5, 2.0]
test_layout = layout2

# Log input once
wandb.log({
    "input_segmentation": wandb.Image(
        test_layout, caption="Input Segmentation"
    )
})

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

# Show input
axes[0].imshow(test_layout)
axes[0].set_title("Input Segmentation", fontsize=12, fontweight="bold")
axes[0].axis("off")

# Test different conditioning scales
for i, scale in enumerate(scales):
    generated = generate_floorplan(
        segmentation_mask=test_layout,
        controlnet_conditioning_scale=scale,
        seed=42,
        show_comparison=False
    )

    # Display in grid
    axes[i + 1].imshow(generated)
    axes[i + 1].set_title(f"Scale={scale}", fontsize=12, fontweight="bold")
    axes[i + 1].axis("off")

    # Log generated image
    wandb.log({
        "conditioning_scale": scale,
        f"generated/scale_{scale}": wandb.Image(
            generated, caption=f"Scale={scale}"
        )
    })

    # Optional: log input vs output comparison
    panel_fig, panel_ax = plt.subplots(1, 2, figsize=(8, 4))
    panel_ax[0].imshow(test_layout)
    panel_ax[0].set_title("Input")
    panel_ax[0].axis("off")
    panel_ax[1].imshow(generated)
    panel_ax[1].set_title(f"Output (scale={scale})")
    panel_ax[1].axis("off")
    plt.tight_layout()

    wandb.log({
        f"comparison/scale_{scale}": wandb.Image(panel_fig)
    })
    plt.close(panel_fig)

# Hide unused subplot
axes[5].axis("off")

plt.suptitle(
    "Conditioning Scale Test: Layout Adherence",
    fontsize=14,
    fontweight="bold"
)
plt.tight_layout()
plt.show()

print("\nExpected Behavior:")
print("   • Scale 0.5: More creative freedom, looser layout")
print("   • Scale 1.0: Balanced (default)")
print("   • Scale 2.0: Stricter layout adherence")

wandb.finish()


## Step 8: Interactive Drawing Interface (GRADIO)

**Upload your own images OR draw layouts interactively!**

This launches a web interface where you can:
- Draw colored room layouts with a brush
- Upload your own segmentation mask images
- Adjust generation parameters
- Get instant results

**The interface will provide a public URL you can access on your phone!**

In [ ]:
# ========================================================================
# IMMEDIATE FIX - Copy and Run This in a New Cell Right Now
# ========================================================================
# This fixes the "controlnet_conditioning_scale must be type float" error
# Just copy this entire block and run it in your current Colab session

def gradio_generate(image_dict, prompt, steps, scale, seed):
    """Fixed Gradio wrapper with proper type conversion."""
    if image_dict is None:
        return None

    try:
        # Extract image
        if isinstance(image_dict, dict) and 'composite' in image_dict:
            img = image_dict['composite']
        else:
            img = image_dict

        # FIX: Convert parameters to correct types
        steps = int(steps)
        scale = float(scale)    # THIS IS THE CRITICAL FIX!
        seed = int(seed)

        # Generate
        result = generate_floorplan(
            segmentation_mask=img,
            prompt=str(prompt),
            num_inference_steps=steps,
            controlnet_conditioning_scale=scale,
            seed=seed,
            show_comparison=False
        )

        return result

    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
        return None

# Now re-launch the Gradio interface with the fixed function
import gradio as gr

with gr.Blocks(title="ControlNet Floorplan Tester") as demo:
    gr.Markdown("""
    # ControlNet Floorplan Generator - Interactive Testing (FIXED)

    **The type error has been fixed! Draw or upload your layouts.**

    1. **Draw**: Use colored brush to draw room layouts
    2. **Upload**: Upload an image (PNG/JPG)
    3. **Generate**: Click button and wait ~10-15 seconds
    """)

    with gr.Row():
        with gr.Column():
            input_image = gr.ImageEditor(
                label="Draw or Upload Segmentation Mask",
                type="pil",
                brush=gr.Brush(
                    colors=[
                        "#9AFF00",  # Living Room (Lime)
                        "#FE9A00",  # Bedroom (Orange)
                        "#9AFF00",  # Kitchen (Lime - same as Living Room)
                        "#009BFF",  # Bathroom (Cyan)
                        "#635047",  # Closet (Brown)
                        "#31639B",  # Corridor (Dark Blue)
                        "#000000"   # Toilet (Black)
                    ],
                    default_size=30
                ),
                height=512
            )

            prompt = gr.Textbox(
                label="Prompt",
                value="a clean architectural floorplan with walls and rooms"
            )

            with gr.Row():
                steps = gr.Slider(10, 50, 20, step=1, label="Steps")
                scale = gr.Slider(0.5, 2.0, 1.0, step=0.1, label="Scale")

            seed = gr.Slider(0, 999999, 42, step=1, label="Seed")
            generate_btn = gr.Button("🚀 Generate Floorplan", variant="primary", size="lg")

        with gr.Column():
            output_image = gr.Image(label="Generated Floorplan", type="pil")

    gr.Markdown("""
    ### 🎨 Color Legend:


    ### Tips:
    - Draw clear rectangular regions
    - Use larger brush for bigger rooms
    - Try different seeds for variations
    - Higher scale = stricter layout adherence
    """)

    generate_btn.click(
        fn=gradio_generate,
        inputs=[input_image, prompt, steps, scale, seed],
        outputs=output_image
    )

# Launch!
demo.launch(share=True, debug=True)

print("\nInterface launched!")
print("Use the public URL to test on your phone or share with others")
print("Upload images OR draw directly on the canvas")
print("Right-click generated images to save them")

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1fd6a47e1f32a95dd6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


##  Step 9: Save Results for Report (Optional)

Generate a comprehensive comparison grid for your AI54 report Section 5.2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

print("Generating report-ready comparison grid...\n")

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Row 1: Different seeds
print("Generating seed variations...")
seeds = [42, 123, 456, 789]
for i, seed in enumerate(seeds):
    if i == 0:
        axes[0, i].imshow(layout3)
        axes[0, i].set_title("Input", fontsize=11, fontweight='bold')
    else:
        generated = generate_floorplan(
            segmentation_mask=layout3,
            seed=seed,
            show_comparison=False
        )
        axes[0, i].imshow(generated)
        axes[0, i].set_title(f"Seed={seed}", fontsize=11, fontweight='bold')
    axes[0, i].axis('off')

# Row 2: Different conditioning scales
print("Generating scale variations...")
scales = [0.5, 1.0, 1.5, 2.0]
for i, scale in enumerate(scales):
    generated = generate_floorplan(
        segmentation_mask=layout3,
        controlnet_conditioning_scale=scale,
        seed=42,
        show_comparison=False
    )
    axes[1, i].imshow(generated)
    axes[1, i].set_title(f"Scale={scale}", fontsize=11, fontweight='bold')
    axes[1, i].axis('off')

plt.suptitle("Model Testing: Seed Variations & Conditioning Scale Exploration",
             fontsize=14, fontweight='bold')
plt.tight_layout()

# Save
output_path = "ai54_section_5_2_comparison_grid.png"
plt.savefig(output_path, dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✅ Saved to: {output_path}")
print("📄 Use this image in your report Section 5.2!")
print("💾 Download it from Colab Files panel (left sidebar)")

---

## 🎉 Testing Complete!

### What You've Done:
1. ✅ Tested model on dataset samples (overfitting check)
2. ✅ Created and tested custom layouts
3. ✅ Checked output diversity with different seeds
4. ✅ Explored conditioning scale effects
5. ✅ Launched interactive drawing interface
6. ✅ Generated comparison grid for report

### Next Steps:
- **Keep the Gradio interface running** to test more layouts
- **Download generated images** for your report
- **Document findings** in Section 5.2
- **Deploy to HuggingFace Spaces** when satisfied!

### For Your Report (Section 5.2):
Include:
- Comparison grid from Step 9
- Example inputs and outputs
- Discussion of layout preservation
- Any limitations observed

---

**Model Repository:** [Qistinasofea/controlnet-floorplan](https://huggingface.co/Qistinasofea/controlnet-floorplan)  
**Dataset:** [Qistinasofea/floorplan-12k-aligned](https://huggingface.co/datasets/Qistinasofea/floorplan-12k-aligned)  
**Training:** 10,000 steps, loss 0.0887, 3h42m on T4 GPU